In [35]:
## start
#print("Hello, World!")

In [1]:
# Import packages
import os
from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import seaborn as sns

import re

#from birdnetlib import Recording
#from birdnetlib.analyzer import Analyzer

#import sqlite3

#import pyarrow.parquet as pq # https://arrow.apache.org/docs/python/parquet.html #error on install

import sys
print(sys.executable)


/opt/anaconda3/envs/nt_bird_webapp_conda_env_20260116/bin/python


In [2]:
# ==========================================
# 1. DIRECTORY CONFIGURATION
# ==========================================

### Project Root Discovery
home_dir = os.path.expanduser('~') # Get the home directory path
PROJECT_ROOT = os.path.join(home_dir, 'Documents/MyProjects/NatureThrive/bio-acoustic-monitor', 'nt-bird-detect') 

#SCRIPT_DIR = os.path.dirname(os.path.abspath("__file__")) # dir of this file
#PROJECT_ROOT = os.path.abspath(os.path.join(SCRIPT_DIR, "../../")) # root dir of project

### Base Data Directories
RAW_DATA_DIR = os.path.join(PROJECT_ROOT, "data/raw")
PROCESSED_DATA_DIR = os.path.join(PROJECT_ROOT, "data/processed")
ANALYTICS_DATA_DIR = os.path.join(PROJECT_ROOT, "data/analytics")

# ==========================================
# 2. Import utility functions
# ==========================================

monitor_name = "wrangcombe_audio1"

#parquet_dir = os.path.join(PROCESSED_DATA_DIR, monitor_name, "recordings_batch.parquet")
parquet_dir = os.path.join(PROCESSED_DATA_DIR, monitor_name, "processing_manifest.parquet")
df_recordings = pd.read_parquet(parquet_dir)
print(df_recordings.head())
print(df_recordings.info())




                      file_name  processed  success         last_updated
0  S4A27301_20250918_045300.wav          1        1  2026-01-21 15:51:45
1  S4A27301_20250918_055302.wav          1        1  2026-01-21 15:52:35
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   file_name     2 non-null      object
 1   processed     2 non-null      int64 
 2   success       2 non-null      int64 
 3   last_updated  2 non-null      object
dtypes: int64(2), object(2)
memory usage: 196.0+ bytes
None
